SCRAPE HTML AND CREATE DATAFRAME
                          


In [114]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'html.parser')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]
df.head()
     


,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


REMOVE NOT ASSIGNED ROWS

In [109]:

toronto_df_dropna = toronto_df[toronto_df.Borough != 'Not assigned'].reset_index(drop=True)
toronto_df_dropna.head()


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


GROUP NEIGHBOURHOOD BT POSTALCODE AND BOROUGH

In [113]:
toronto_df_grouped = toronto_df_dropna.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [ ]:
DEAL WITH NOT ASSIGNED NEIGHBOURHOOD AND SHAPE DATAFRAME

In [112]:
To_neigh_rows = toronto_df_grouped.Neighbourhood == 'Not assigned'
toronto_df_grouped.loc[To_neigh_rows, 'Neighbourhood'] = toronto_df_grouped.loc[To_neigh_rows, 'Borough']
toronto_df_grouped[To_neigh_rows]
toronto_df_grouped.shape

(103, 3)